<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLN-131121_Regres_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perceptron -- neuron with an activation function.
Inputs and their weights are numbered 0 to(n-1), bias is n

In [1]:
import numpy as np

# Single Neuron

In [2]:
class Neuron:
    """A single neuron with an activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0.
          activ:  The activation function: linear (default), relu, sigmoid."""

    def __init__(self, inputs, bias = 1.0, activ = 'linear'):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias) and random initial weights.""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias
        self.activ = activ

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        if self.activ == 'linear':
          return sum
        if self.activ == 'sigmoid':
          return self.sigmoid(sum)
        if self.activ == 'relu':
          return self.relu(sum)

    def set_weights(self, w_init):
        """Overrides the np.random.rand() weights and the bias weight."""
        # w_init is a list of floats. Organize it as you'd like.
        self.weights=np.array(w_init)

    def set_activ(self, activ):
        """Overrides the 'linear' activation function."""
        self.activ = activ

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))
    
    def relu(self, x):
        # return the output of the relu function applied to x
        if x >= 0:
          return x
        return 0

In [3]:
neu=Neuron(inputs=2)
neu.set_weights([10,10,-15]) 
# neu.set_activ('sigmoid')
neu.run([1,1])

5.0

## AND gate

In [4]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-15]) #AND gate

print("AND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


AND Gate:
0 0 = 0.0000003059
0 1 = 0.0066928509
1 0 = 0.0066928509
1 1 = 0.9933071491


In [5]:
#proverka
# a=[]
# a.append(Perceptron(inputs=2))
# a[1].weights

## OR gate

In [6]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([10,10,-5]) #OR gate

print("OR Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))


OR Gate:
0 0 = 0.0066928509
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.9999996941


## NAND gate

In [7]:
neuron = Neuron(inputs=2, activ='sigmoid')
neuron.set_weights([-10,-10,15]) #NAND gate

print("NAND Gate:")
print ("0 0 = {0:.10f}".format(neuron.run([0,0])))
print ("0 1 = {0:.10f}".format(neuron.run([0,1])))
print ("1 0 = {0:.10f}".format(neuron.run([1,0])))
print ("1 1 = {0:.10f}".format(neuron.run([1,1])))

NAND Gate:
0 0 = 0.9999996941
0 1 = 0.9933071491
1 0 = 0.9933071491
1 1 = 0.0066928509


# Multilayer neuron

In [8]:
class MultiLayerNeuron:     
    """A multilayer neuron class that uses the Neuron class above.
       Builds a list of neurons with the specific activation function.
       The activation function may be modified later using the set_activ method.
       For example: mln.network[layer][neuron].set_activ('linear').
       Attributes:
          layers:  A list with the number of neurons per layer. Including the input (first) and the output (last) layers.
          bias:    The bias term. The same bias is used for all neurons.
          eta:     The learning rate.
          activ:   The activation function: linear (default), relu, sigmoid."""

    def __init__(self, layers, bias = 1.0, eta = 0.5, activ='linear'):
        """Return a new MLP object with the specified parameters.""" 
        self.layers = np.array(layers,dtype=object)
        self.bias = bias
        self.eta = eta
        self.network = [] # The list of lists of neurons (perceptrons).
        self.values = []  # The list of lists of neurons' (perceptrons') output values.
        self.d = []       # The list of lists of error terms (lowercase deltas)
        self.activ = activ

        # 2 nested loops to create neurons layer by layer
        for i in range(len(self.layers)): # outer loop iterates on each layer
            self.values.append([]) #The new list of values will be filled with zeros, for every neuron in the layer. 
            self.values[i] = [0.0 for j in range(self.layers[i])]
            self.d.append([])
            self.d[i] = [0.0 for j in range(self.layers[i])]                        
            self.network.append([])
            if i > 0:      #network[0] is the input layer, so it has no neurons
                for j in range(self.layers[i]): # inner loop iterates on each neuron in a layer
                    neur=Neuron(inputs = self.layers[i-1], bias = self.bias, activ = self.activ) # 
                    self.network[i].append(neur) # adding j perceptrons
        
        self.network = np.array([np.array(x) for x in self.network],dtype=object) #transforms list of lists to numpy array
        self.values = np.array([np.array(x) for x in self.values],dtype=object)
        self.d = np.array([np.array(x) for x in self.d],dtype=object)

    def set_weights(self, w_init): # set_weights of the MultiLayer class
        """Set the weights. 
           w_init is a list of lists with the weights for all but the input layer."""
        for i in range(len(w_init)):
            for j in range(len(w_init[i])):
                self.network[i+1][j].set_weights(w_init[i][j]) # set_weights for each perceptron i

    def set_activ(self, activ):
        """Set the activation function to every neurons."""
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                self.network[i][j].set_activ(activ) # set_activ for each neuron
    
    def set_output_activ(self, activ):
        """Set the activation function to the last (output) neurons."""
        i = len(self.network)-1
        for j in range(self.layers[i]):
            self.network[i][j].set_activ(activ) 

    def printWeights(self):
        """Displays a summary of weights and activation functions per layer and neuron."""
        print()
        print('Layer 0 is the Input Layer')
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):
                print("Layer",i,"Neuron",j,":",self.network[i][j].weights,self.network[i][j].activ)
        print()

    def run(self, x):
        """Feed a sample x into the MultiLayer Neuron."""
        x = np.array(x,dtype=object)
        self.values[0] = x
        for i in range(1,len(self.network)):
            for j in range(self.layers[i]):  
                self.values[i][j] = self.network[i][j].run(self.values[i-1]) #runs preceptrons with the previous outputs
        return self.values[-1]

    def bp_classif(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative of the sigmoid function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        self.d[-1] = outputs * (1 - outputs) * (error) # derivative of the SIGMOID function 
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k]               
                self.d[i][h] = self.values[i][h] * (1-self.values[i][h]) * fwd_error # derivative of the SIGMOID function
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    if k==self.layers[i-1]:
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] # applying the delta rule
                    self.network[i][j].weights[k] += delta
        return MSE

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))

    def deriv(self, value, i, j=0):
        if self.network[i][j].activ == 'linear':
          # print ('lin')
          return 1
        if self.network[i][j].activ == 'sigmoid':
          # print ('sig')
          return self.sigmoid(value)*(1-self.sigmoid(value))
        if self.network[i][j].activ == 'relu':
          if value > 0:
            # print ('re>')
            return 1
          else:
            # print ('re<')
            return 0

    def bp_regres(self, x, y):
        """Run a single (x,y) pair with the backpropagation algorithm - Gradient Descent.
        Uses the derivative according each neuron's activation function."""
        x = np.array(x,dtype=object)
        y = np.array(y,dtype=object)
        # STEP 1: Feed a sample to the network 
        outputs = self.run(x)
        # STEP 2: Calculate the MSE
        error = 2*(y - outputs) # A list of outputs
        MSE = sum( error ** 2) / self.layers[-1] 
        # ∂MSE/∂weight=∂MSE/∂output*∂output/∂weight
        # STEP 3: Calculate the OUTPUT error terms
        # ∂MSE/∂output -- depends on neuron's activation function
        for j in range (len(outputs)):
            self.d[-1][j] = self.deriv(outputs[j], len(self.network)-1) * error
        # STEP 4: Calculate the error term of EACH UNIT on each layer
        for i in reversed(range(1,len(self.network)-1)):
            for h in range(len(self.network[i])):
                fwd_error = 0.0
                for k in range(self.layers[i+1]): 
                    fwd_error += self.network[i+1][k].weights[h] * self.d[i+1][k] 
                self.d[i][h] = self.deriv(self.values[i][h], i, h) * fwd_error
        # STEPS 5 & 6: Calculate the deltas and update the weights
        for i in range(1,len(self.network)): # runs on layers
            for j in range(self.layers[i]): # runs on neurons
                for k in range(self.layers[i-1]+1): # runs on inputs. +1 for bias
                    # output=sum(weight*value)+bias*bias_weight
                    if k==self.layers[i-1]:
                        # ∂output/∂bias_weight=bias
                        delta = self.eta * self.d[i][j] * self.bias
                    else:
                        # ∂output/∂weight=value
                        delta = self.eta * self.d[i][j] * self.values[i-1][k] 
                    self.network[i][j].weights[k] += delta # applying the delta rule
        return MSE

In [21]:
class Regres:
    def __init__(self, layers, epochs, bias=1.0, eta=0.5, ):
        self.layers = layers
        self.bias = bias
        self.eta = eta
        self.epochs = epochs      
        self.network = MultiLayerNeuron(layers=layers, bias=bias, eta=eta)

    def set_weights(self, w_init):
        self.network.set_weights(w_init)

    def set_hidden_activ(self, activ):
        self.network.set_activ(activ)
        self.network.set_output_activ('linear')

    def fit(self, X, y):
        for i in range(self.epochs):
            MSE = 0.0
            for j in range (len(X)):
                MSE +=  self.network.bp_regres(X[j],y[j])
            MSE = MSE / len(X)
            # if(i%200 == 0):
            #     print ('MSE=',MSE)
        self.network.printWeights()        



## XOR gate=(OR+NAND)+AND

In [10]:
#test code
mln1 = MultiLayerNeuron(layers=[2,2,1])  #mln1
mln1.set_weights([[[-10,-10,15],[15,15,-10]],[[10,10,-15]]])
mln1.set_activ('sigmoid') #linear is by default

mln1.printWeights()
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln1.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln1.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln1.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln1.run([1,1])[0]))


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-10 -10  15] sigmoid
Layer 1 Neuron 1 : [ 15  15 -10] sigmoid
Layer 2 Neuron 0 : [ 10  10 -15] sigmoid

XOR Gate:
0 0 = 0.0066958493
0 1 = 0.9923558642
1 0 = 0.9923558642
1 1 = 0.0071528098


## Training 

In [11]:
mln2 = MultiLayerNeuron(layers=[2,2,1])
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

1.162005056453476
1.068922056333509
0.8444554805736075
0.7444174573301721
0.7240191178119046
0.7158609121614526
0.7115688255474046
0.7089542593853687
0.7072084220597152
0.7059667333979731

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-3.84215667 -3.92137525 -2.02025965] sigmoid
Layer 1 Neuron 1 : [-6.8894564  -6.96920638  0.64427798] sigmoid
Layer 2 Neuron 0 : [-1.36404333 -5.77054227  0.66678178] sigmoid

XOR Gate:
0 0 = 0.0363758529
0 1 = 0.6576613605
1 0 = 0.6574026887
1 1 = 0.6607625815


# Example1 -- weights

In [12]:
print ("1 1 =",mln1.run([1,1]))
print (mln1.network[1][0].weights) # network is list of lists of perceptrons. Each has attribute "weights"

1 1 = [0.00715281]
[-10 -10  15]


In [13]:
mln1=MultiLayerNeuron(layers=[2,2,1])
mln1.set_activ('relu')
mln1.printWeights()

mln1.set_output_activ('sigmoid') # setting the output activ func
mln1.printWeights()

mln1.network[1][0].set_activ('linear') # changing specific activ func
mln1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.26281132  0.92146892 -0.23285572] relu
Layer 1 Neuron 1 : [-0.38051282  0.34744814 -0.68990812] relu
Layer 2 Neuron 0 : [ 0.36850397 -0.20721248  0.33324631] relu


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.26281132  0.92146892 -0.23285572] relu
Layer 1 Neuron 1 : [-0.38051282  0.34744814 -0.68990812] relu
Layer 2 Neuron 0 : [ 0.36850397 -0.20721248  0.33324631] sigmoid


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-0.26281132  0.92146892 -0.23285572] linear
Layer 1 Neuron 1 : [-0.38051282  0.34744814 -0.68990812] relu
Layer 2 Neuron 0 : [ 0.36850397 -0.20721248  0.33324631] sigmoid



# Example 2 -- sigmoid, bp_classif, XOR

In [14]:
mln2 = MultiLayerNeuron(layers=[2,2,1], eta=0.5)
mln2.set_activ('sigmoid') #linear is by default
print("\nTraining Neural Network as an XOR Gate...\n")
for i in range(2000):
    MSE = 0.0
    MSE += mln2.bp_classif([0,0],[0])
    MSE += mln2.bp_classif([0,1],[1])
    MSE += mln2.bp_classif([1,0],[1])
    MSE += mln2.bp_classif([1,1],[0])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln2.printWeights()
    
print("XOR Gate:")
print ("0 0 = {0:.10f}".format(mln2.run([0,0])[0]))
print ("0 1 = {0:.10f}".format(mln2.run([0,1])[0]))
print ("1 0 = {0:.10f}".format(mln2.run([1,0])[0]))
print ("1 1 = {0:.10f}".format(mln2.run([1,1])[0]))


Training Neural Network as an XOR Gate...

1.1077634883720682
1.0767804012498225
1.0288339826928434
0.7064916029241728
0.1652663199555989
0.031803601889053686
0.015964797716531372
0.010423588327075115
0.007666574330954245
0.0060332982465207945

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [-6.01743145 -6.24239021  2.39357364] sigmoid
Layer 1 Neuron 1 : [-4.26030907 -4.2940128   6.32748451] sigmoid
Layer 2 Neuron 0 : [-8.70609008  8.4860556  -3.95029225] sigmoid

XOR Gate:
0 0 = 0.0305586131
0 1 = 0.9668189379
1 0 = 0.9663105303
1 1 = 0.0421083457


# Example 3 -- linear activs, bp_regres

In [15]:
mln3 = MultiLayerNeuron(layers=[3,1], eta=0.3)
# mln3.set_weights([[[4.9,-0.9,2.1,0.2]]])
# mln3.printWeights()
print("\nTraining Neural Network...\n")
for i in range(400):
    MSE = 0.0
    MSE += mln3.bp_regres([0.7759,	0.1104,	0.9977,],[5.764286995])
    MSE += mln3.bp_regres([0.9692,	0.6961,	0.8483,],[5.84646758])
    MSE += mln3.bp_regres([0.0265,	0.399,	0.5375,],[0.808633075])
    MSE += mln3.bp_regres([0.7694,	0.5051,	0.2542,],[3.850298589])
    MSE = MSE / 4
    if(i%200 == 0):
        print (MSE)

mln3.printWeights()




Training Neural Network...

41.83771587934031
9.13686480516414e-13

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 4.99973369e+00 -9.99625006e-01  1.99991681e+00  3.52253295e-05] linear



# Example 4 -- sigmoid, bp_regres

In [16]:
# generating output values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.3)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[-1,2,3]]])
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1], eta=0.3)
mln4.set_activ('sigmoid')
mln4.set_output_activ('linear')

for i in range(1000):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0  1 -2  1] sigmoid
Layer 1 Neuron 1 : [ 1 -2  0  1] sigmoid
Layer 2 Neuron 0 : [-1  2  3] linear



'original values:'

[3.5813200792821274,
 3.731058578630005,
 3.731058578630005,
 3.119202922022118,
 3.880797077977882,
 2.2563920540063265]


MSE= 19.225511948354722
MSE= 0.002878639855366375
MSE= 0.003006474219359063
MSE= 0.003099113283803063
MSE= 0.0031570097934711747

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.44951447 -1.0252568   0.55618316 -0.21638918] sigmoid
Layer 1 Neuron 1 : [ 0.50770123 -1.24712768  0.36626488  0.26070414] sigmoid
Layer 2 Neuron 0 : [1.69497346 2.00748397 1.82438623] linear



'predicted values:'

[3.5463796321471746,
 3.688954397641268,
 3.7143813785299535,
 3.120699183160717,
 3.9003565246794967,
 2.2567129003141964]

# Example 5 -- various, bp_regres

In [17]:
mln4 = MultiLayerNeuron(layers=[3,2,1,1]) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')
mln4.set_weights([[[0,1,-2,1],[1,-2,0,1]],[[1,0,1]],[[-1,2]]]) # setting weights
mln4.printWeights()

y4=[] # generating output values
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('original values:',y4)
print ()

# training model and predicting values
mln4 = MultiLayerNeuron(layers=[3,2,1,1], eta=0.3) # generating MLN with random weights
mln4.set_activ('relu')
mln4.network[2][0].set_activ('sigmoid') # changing specific activ func
mln4.set_output_activ('linear')

for i in range(400):
    MSE = 0.0
    MSE += mln4.bp_regres([2,1,0],y4[0])
    MSE += mln4.bp_regres([-1,0,1],y4[1])
    MSE += mln4.bp_regres([0,0,0],y4[2])
    MSE += mln4.bp_regres([1,1,0],y4[3])
    MSE += mln4.bp_regres([-1,-1,-1],y4[4])
    MSE += mln4.bp_regres([-1,1,-1],y4[5])
    MSE = MSE / 6
    if(i%200 == 0):
        print ('MSE', i, '=',MSE)
mln4.printWeights()

y4=[]
y4.append(mln4.run([2,1,0])[0])
y4.append(mln4.run([-1,0,1])[0])
y4.append(mln4.run([0,0,0])[0])
y4.append(mln4.run([1,1,0])[0])
y4.append(mln4.run([-1,-1,-1])[0])
y4.append(mln4.run([-1,1,-1])[0])
display ('predicted values:',y4)
print ()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0  1 -2  1] relu
Layer 1 Neuron 1 : [ 1 -2  0  1] relu
Layer 2 Neuron 0 : [1 0 1] sigmoid
Layer 3 Neuron 0 : [-1  2] linear



'original values:'

[1.0474258731775667,
 1.2689414213699952,
 1.1192029220221178,
 1.0474258731775667,
 1.0474258731775667,
 1.0066928509242847]


MSE 0 = 1.3066363786449082
MSE 200 = 0.060899366608554856

Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.96884029 -0.94395995  0.61567453 -0.02506902] relu
Layer 1 Neuron 1 : [ 0.7235423   0.50754241 -0.3708328  -0.34084255] relu
Layer 2 Neuron 0 : [ 0.58468073 -0.28708315  0.29084603] sigmoid
Layer 3 Neuron 0 : [1.29967495 0.27059458] linear



'predicted values:'

[1.046790390983952,
 1.0142728014870843,
 1.0142728014870843,
 0.9318915679522588,
 1.0142728014870843,
 1.0142728014870843]

# Example 6 -- https://mmuratarat.github.io/2020-01-09/backpropagation

In [18]:
# generating output values
mln6 = MultiLayerNeuron(layers=[5,3,1], eta=.5)
mln6.set_activ('sigmoid')
mln6.set_output_activ('linear')
mln6.set_weights([[[.19,.33,.4,.51,.54,.1],
                   [.55,.16,.35,.85,.49,.1],
                   [.76,.97,.7,.85,.57,.1]],
                  [[.1,.03,-.17,.1]]
                  ]) # setting initial weights
mln6.printWeights()

# training
for i in range(1):
    MSE = 0.0
    MSE += mln6.bp_regres([.5,.1,1,0,0],[0.1])
    MSE += mln6.bp_regres([.3,.2,0,1,0],[.6])
    MSE += mln6.bp_regres([.7,.9,0,0,1],[.4])
    MSE += mln6.bp_regres([.8,.1,1,0,0],[.1])
    MSE = MSE / 4

mln6.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.19 0.33 0.4  0.51 0.54 0.1 ] sigmoid
Layer 1 Neuron 1 : [0.55 0.16 0.35 0.85 0.49 0.1 ] sigmoid
Layer 1 Neuron 2 : [0.76 0.97 0.7  0.85 0.57 0.1 ] sigmoid
Layer 2 Neuron 0 : [ 0.1   0.03 -0.17  0.1 ] linear


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 0.05768057  0.24517142  0.30611063  0.52237603  0.45342795 -0.06808539] sigmoid
Layer 1 Neuron 1 : [ 0.41131404  0.07909778  0.24408963  0.85570547  0.41060858 -0.07959631] sigmoid
Layer 1 Neuron 2 : [ 0.5723622   0.91183233  0.50452277  0.83796066  0.52976436 -0.14775221] sigmoid
Layer 2 Neuron 0 : [0.60611773 0.63624137 0.50097927 1.03498116] linear

